In [1]:
import json
from PIL import Image
from IPython.display import display
import numpy as np
import pandas as pd
# from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel
# from torch.nn.utils.rnn import pad_sequence
# import torch

import nltk, csv, collections
# nltk.download('punkt')
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.svm import LinearSVC
from sklearn.metrics import precision_recall_fscore_support
from scipy.sparse import hstack
import spacy

from transformers import pipeline, AutoModelWithLMHead, AutoTokenizer

# nlp = spacy.load('en_core_web_sm')

In [2]:
# importing the data
datafolder = '../../data/hateful_memes/'
train = datafolder+'train.jsonl'
test = datafolder+'test_seen.jsonl'
dev = datafolder+'dev_seen.jsonl'
# Load the data from the JSON file
df_train = pd.read_json(train, lines = True)
df_dev = pd.read_json(dev, lines = True)
df_test = pd.read_json(test, lines = True)

In [3]:
def preprocess_row(row):
    text = row['text']
    doc = nlp(text)
    tokens = []
    for token in doc:
        pos = token.pos_
        lemma = token.lemma_
        tokens.append((token.text, lemma, pos))
    row['tokens'] = " ".join([t[0] for t in tokens])
    row['lemmas'] = " ".join([t[1] for t in tokens])
    row['upos'] = " ".join([t[2] for t in tokens])
    return row

In [4]:
df_train = df_train.apply(preprocess_row, axis=1)
df_dev = df_dev.apply(preprocess_row, axis=1)
df_test = df_test.apply(preprocess_row, axis=1)


# advanced SVM 
#### - pos_fw_emo = representation of the text through POS tags, function words, and emotion words (from this representation n-grams (n=1-3) are built, see vectorize below)
#### - count = number of emotion words in a text
#### - emotion_associations = emotion associations from the NRC emotion lexicon

In [27]:
# load the NRC emotion lexicon into a dictionary with emotion words and corresponding associations
lexicon = '../../data/hateful_memes/nrc-lexicon-en.txt' # path to the NRC emotion lexicon
emotions = {}
for line in open(lexicon).read().split('\n'):	
    emotion_word = line.split('\t')[0]
    emotion = line.split('\t')[1]
    association = line.split('\t')[2]
    if association == "1":
        if emotion_word in emotions:
            emotions[emotion_word].append(emotion)
        else:
            emotions[emotion_word] = [emotion] 

list(emotions.items())[:3] # print first 3 entries

[('smut', ['disgust', 'fear', 'negative']),
 ('expletive', ['anger', 'negative']),
 ('greeting', ['positive', 'surprise'])]

In [64]:
# - pos_fw_emo = representation of the text through POS tags, function words, and emotion words (from this representation n-grams (n=1-3) are built, see vectorize below)
# - count = number of emotion words in a text
# - emotion_associations = emotion associations from the NRC emotion lexicon
# - Sentiment score = using siebert/sentiment-roberta-large-english from huggingface we retrieve the sentiment score of the whole sentence
# - Intent = using mrm8488/t5-base-finetuned-e2m-intent we retrieve the intent of the sentence 

fw_list = ['ADP', 'AUX', 'CCONJ', 'DET', 'NUM', 'PART', 'PRON', 'SCONJ'] # POS tags that correspond to function words

tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-e2m-intent")
intent_model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-e2m-intent")
def get_intent(event):
    input_text = "%s </s>" % event
    features = tokenizer([input_text], return_tensors='pt')

    output = intent_model.generate(input_ids=features['input_ids'], 
               attention_mask=features['attention_mask'])

    return tokenizer.decode(output[0])[6:-4]

sentiment_analysis = pipeline("sentiment-analysis",model="siebert/sentiment-roberta-large-english")
def senti(text):
    output = sentiment_analysis(text)[0]
    if output['label'] == 'POSITIVE':
        return output['score']
    else:
        return 1 - output['score']
    
def get_feats_en(upos, lemmas, text):	
    pos_fw_emo = []
    count = 0
    emotion_associations = []
    sentiment_score = senti(text)
    intent = get_intent(text)
    for i, lemma in enumerate(lemmas.split()):		
        if lemma.lower() in emotions:
            pos_fw_emo.append(lemma)
            count += 1
            emotion_associations.append(emotions[lemma.lower()])     
        else:
            if upos.split()[i] in fw_list:
                pos_fw_emo.append(lemma)
            else:
                pos_fw_emo.append(upos.split()[i])
    emotion_associations = [emo for sublist in emotion_associations for emo in sublist]
    return pd.Series([' '.join(pos_fw_emo), count, ' '.join(emotion_associations), sentiment_score, intent])

df_train[['pos_fw_emo', 'count', 'emotion_associations', 'sentiment_score', 'intent']] = df_train.apply(lambda x: get_feats_en(x['upos'], x['lemmas'], x['text']), axis=1) 
df_dev[['pos_fw_emo', 'count', 'emotion_associations', 'sentiment_score', 'intent']] = df_dev.apply(lambda x: get_feats_en(x['upos'], x['lemmas'], x['text']), axis=1) 
df_test[['pos_fw_emo', 'count', 'emotion_associations', 'sentiment_score', 'intent']] = df_test.apply(lambda x: get_feats_en(x['upos'], x['lemmas'], x['text']), axis=1) 

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
C:\Users\Hisha\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\generation\utils.py:1288: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
C:\Users\Hisha\AppData\Local\Pr

In [84]:
# df_train['sentiment_score'] = df_train.apply(lambda x: senti(x['text']), axis=1)
# df_dev['sentiment_score'] = df_dev.apply(lambda x: senti(x['text']), axis=1)
df_test['sentiment_score'] = df_test.apply(lambda x: senti(x['text']), axis=1)

In [85]:
df_train.to_csv(datafolder+'train_with_features.csv', index=False)
df_dev.to_csv(datafolder+'dev_with_features.csv', index=False)
df_test.to_csv(datafolder+'test_with_features.csv', index=False)

In [2]:
# importing the data
datafolder = '../../data/hateful_memes/'
train = datafolder+'train_with_features.csv'
test = datafolder+'test_with_features.csv'
dev = datafolder+'dev_with_features.csv'
df_train = pd.read_csv(train, keep_default_na=False)
df_dev = pd.read_csv(dev, keep_default_na=False)
df_test = pd.read_csv(test, keep_default_na=False)

In [3]:
vectorizer1 = CountVectorizer(tokenizer=lambda x: x.split(), analyzer='word', ngram_range=(1, 3)) # to build n-grams (n=1-3) from the pos_fw_emo representation
vectorizer2 = CountVectorizer(tokenizer=lambda x: x.split(), analyzer='word', ngram_range=(1, 1)) # unigrams of emotion associations
vectorizer3 = CountVectorizer(tokenizer=lambda x: x.split(), analyzer='word', ngram_range=(1, 1)) # unigrams of tokens (BoW)
vectorizer4 = CountVectorizer(tokenizer=lambda x: x.split(), analyzer='word', ngram_range=(1, 4)) # unigrams of intent (BoW)


# combine the features
X_train = hstack((vectorizer1.fit_transform(df_train.pos_fw_emo), vectorizer2.fit_transform(df_train.emotion_associations), df_train[['count']].values, \
                df_train[['sentiment_score']].values, vectorizer3.fit_transform(df_train.tokens), vectorizer4.fit_transform(df_train.intent)), format='csr') 

X_dev = hstack((vectorizer1.transform(df_dev.pos_fw_emo), vectorizer2.transform(df_dev.emotion_associations), df_dev[['count']].values, \
                df_dev[['sentiment_score']].values, vectorizer3.transform(df_dev.tokens), vectorizer4.transform(df_dev.intent) ), format='csr') 

X_test = hstack((vectorizer1.transform(df_test.pos_fw_emo), vectorizer2.transform(df_test.emotion_associations), df_test[['count']].values, \
                df_test[['sentiment_score']].values, vectorizer3.transform(df_test.tokens), vectorizer4.transform(df_test.intent) ), format='csr') 

Y_train = df_train.label.values
Y_dev = df_dev.label.values
Y_test = df_test.label.values

In [4]:
# clf_svc = LinearSVC(max_iter=100000) # parameter C was selected based on grid search
clf_svc = LinearSVC(max_iter=100000, C = 10,random_state =456)
clf_svc.fit(X_train, Y_train)
Y_pred = clf_svc.predict(X_dev)

In [5]:
results = pd.DataFrame(
    [list(precision_recall_fscore_support(Y_dev, Y_pred, average='macro')[:3])],
    columns=['precision', 'recall', 'F1'])
results

,precision,recall,F1
0,0.542501,0.533197,0.50764


In [6]:
Y_pred2 = clf_svc.predict(X_test)

df_dev['Advanced_svm_linear_100K_C10'] = Y_pred
df_test['Advanced_svm_linear_100K_C10'] = Y_pred2

df_dev.to_csv(datafolder+'dev_with_features.csv', index=False)
df_test.to_csv(datafolder+'test_with_features.csv', index=False)